In [1]:
from src.model_train import ModelTrainer, validate_folds
import pandas as pd

from cashe import get_training_inputs, drop_nomodel_columns, get_train_test_split
from src.data_local import MergedDataPreprocessing,read_last_date, append_last_line
from src.lstm_encoder import LSTMEmbedding
from src.model_app import ModelApplication

In [2]:
df_train, df_test = get_train_test_split()

In [3]:
df_train.head()

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,Episode_Key,ICD10,Diagnosis_Key
398359,9643,2024-01-01 12:33:25,194135,134.40,Consultant Consultation,1.0,120.00,120.00,24.0,96.00,...,FILE,N,IQ,2023-12-07 00:00:00,2023-12-21 00:00:00,2023-12-01,2023-12-10,1_1346536-2,J93,1_1111225808
398360,9643,2024-01-01 12:38:12,194136,138.00,Consultation,1.0,120.00,120.00,0.0,120.00,...,FILE,N,H,2023-12-06 00:00:00,2023-12-20 00:00:00,2023-12-01,2023-12-10,1_1343530-4,K02,1_1111225570
398594,1,2024-01-02 01:04:02,194152,11.38,TRANDATE INJ. 100 MG.-AMP.20ML,1.0,11.98,11.98,0.0,11.98,...,FILE,N,IQ,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
398595,1,2024-01-02 01:04:02,194152,15346.75,Elective classical caesarean section,1.0,15700.00,15700.00,0.0,15700.00,...,FILE,N,IQ,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
398596,1,2024-01-02 01:04:02,194152,575.00,Routine preoperative anaes assessment,1.0,500.00,500.00,0.0,500.00,...,FILE,N,IQ,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440


In [4]:
read_last_date()

'2024-04-30'

In [5]:
X_train, y_train, X_test, y_test = get_training_inputs(df_train, df_test)

In [6]:
preprocessing_train = MergedDataPreprocessing(X_train)
X_train_prep = preprocessing_train.columns_prep(service_encoding=False)

preprocessing_test = MergedDataPreprocessing(X_test)
X_test_prep = preprocessing_test.columns_prep(service_encoding=False)

E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[column] = self.df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:280: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:275: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `resul

In [7]:
lstm_embedding = LSTMEmbedding()

X_train_encoded = preprocessing_train.column_embedding(X_train_prep)
X_test_encoded = preprocessing_train.column_embedding(X_test_prep)

In [8]:
X_train = drop_nomodel_columns(X_train_encoded)
X_test  = drop_nomodel_columns(X_test_encoded)

In [9]:
model_runner = ModelApplication(X_train= X_train,y_train= y_train['OUTCOME'],X_test= X_test,y_test= y_test['OUTCOME'])

In [12]:
model_runner.get_model(force_retrain=False,model_type='outcome')

Training is done


In [13]:
## runtime 1
model_runner.evaluate_model()

{'XGBoost': {'Accuracy': 0.77,
  'Precision': 0.76,
  'Recall': 0.82,
  'F1 Score': 0.79}}

In [14]:
## runtime 2
model_runner.evaluate_model()

{'XGBoost': {'Accuracy': 0.77,
  'Precision': 0.76,
  'Recall': 0.82,
  'F1 Score': 0.79}}

## Columns removal comparison

In [10]:
## before the high correlation appliance
model_runner = ModelApplication(X_train=X_train, y_train=y_train['OUTCOME'], X_test=X_test, y_test=y_test['OUTCOME'])
model_runner.get_model(force_retrain=False, model_type='outcome')
model_runner.evaluate_model()

XGBoost Model is loading from disk..


{'XGBoost': {'Accuracy': 0.77,
  'Precision': 0.76,
  'Recall': 0.81,
  'F1 Score': 0.78}}

In [17]:
features_imp = list(model_runner.get_feature_importance())
pd.DataFrame({"Column name":list(X_train.columns),"Importance":features_imp}).sort_values(ascending=False,by=['Importance']).to_excel('FeatureImportance_beforedrop.xlsx',index=False)

In [21]:
cols_drop = ['CLAIM_INVOICE_NO','STAT_INVOICE_NO','CLAIM_INVOICE_NO','HIS_INSURANCE_CODE','CREATED_BY_USER','TOTAL_NET_AMOUNT',
             'TOTAL_NET_VAT_AMOUNT','TOTAL_CLAIMED_AMOUNT','LINE_CLAIMED_AMOUNT','CO_INSURANCE','NET_AMOUNT','UNIT_PRICE',
             'CO_PAY']

cols_kept = [col for col in X_train.columns if col not in cols_drop]

## after the high correlation appliance
model_runner = ModelApplication(X_train=X_train[cols_kept], y_train=y_train['OUTCOME'],
                                X_test=X_test[cols_kept], y_test=y_test['OUTCOME'])

model_runner.get_model(force_retrain=True, model_type='outcome')
model_runner.evaluate_model()

Training is done


{'XGBoost': {'Accuracy': 0.77,
  'Precision': 0.75,
  'Recall': 0.81,
  'F1 Score': 0.78}}

In [23]:
features_imp = list(model_runner.get_feature_importance())
pd.DataFrame({"Column name":list(cols_kept),"Importance":features_imp}).sort_values(ascending=False,by=['Importance']).to_excel('FeatureImportance_afterdrop.xlsx',index=False)

## Other old trials

In [14]:
## Comparison between the embedding vector added columns
print('The result before embedding the Item_Name column are:\n',res_before,'\nThe results after the embedding:\n',res)

The result before embedding the Item_Name column are:
 {'Decision Tree': {'Accuracy': 0.67, 'Precision': 0.66, 'Recall': 0.64}, 'LightGBM': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.71}, 'SGD Classifier': {'Accuracy': 0.52, 'Precision': 0.0, 'Recall': 0.0}, 'XGBoost': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.73}, 'Neural Network': {'Accuracy': 0.56, 'Precision': 0.55, 'Recall': 0.44}} 
The results after the embedding:
 {'Decision Tree': {'Accuracy': 0.67, 'Precision': 0.66, 'Recall': 0.63}, 'LightGBM': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.71}, 'SGD Classifier': {'Accuracy': 0.48, 'Precision': 0.48, 'Recall': 0.9}, 'XGBoost': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.73}, 'Neural Network': {'Accuracy': 0.55, 'Precision': 0.57, 'Recall': 0.28}}


In [16]:
## Feature importance Case Study
dt_tran = model_runner.get_decision_tree_feature_importance()
dt_feats = pd.DataFrame(list(dt_tran),columns_needed)

dt_feats[0].sort_values(ascending=False)

item_Price                     0.323421
transaction_PatientAge         0.313583
item_Sequence                  0.191764
transaction_DiagnosisIds       0.114149
transaction_PatientEnGender    0.044771
item_RequestQuantity           0.012312
Name: 0, dtype: float64

In [17]:
gbm_tran = model_runner.get_lightgbm_feature_importance()
gbm_feats = pd.DataFrame(list(gbm_tran),columns_needed)

gbm_feats[0].sort_values(ascending=False)

item_Price                     0.410000
transaction_PatientAge         0.255333
transaction_DiagnosisIds       0.137667
item_Sequence                  0.135333
item_RequestQuantity           0.031000
transaction_PatientEnGender    0.030667
Name: 0, dtype: float64

In [18]:
nn_tran = model_runner.get_neural_network_feature_importance()
nn_feats = pd.DataFrame(list(nn_tran),columns_needed)

nn_feats[0].sort_values(ascending=False)

item_Price                     7.984434e-02
item_RequestQuantity           5.211179e-02
item_Sequence                  3.358094e-02
transaction_DiagnosisIds       2.544493e-02
transaction_PatientAge         2.421929e-02
transaction_PatientEnGender    1.265203e-53
Name: 0, dtype: float64

## K-Fold Analysis

In [2]:
df_original = get_input()
preprocessing = MergedDataPreprocessing(df_original)

In [11]:
df_acc = validate_folds(df_original)



LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7437, number of negative: 7506
[LightGBM] [Info] Total Bins 1303
[LightGBM] [Info] Number of data points in the train set: 14943, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497691 -> initscore=-0.009235
[LightGBM] [Info] Start training from score -0.009235


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7480, number of negative: 7714
[LightGBM] [Info] Total Bins 1317
[LightGBM] [Info] Number of data points in the train set: 15194, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.492300 -> initscore=-0.030804
[LightGBM] [Info] Start training from score -0.030804


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7558, number of negative: 7626
[LightGBM] [Info] Total Bins 1298
[LightGBM] [Info] Number of data points in the train set: 15184, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497761 -> initscore=-0.008957
[LightGBM] [Info] Start training from score -0.008957


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7480, number of negative: 7518
[LightGBM] [Info] Total Bins 1300
[LightGBM] [Info] Number of data points in the train set: 14998, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498733 -> initscore=-0.005067
[LightGBM] [Info] Start training from score -0.005067


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [12]:
df_acc

,Decision Tree,LightGBM,SGD Classifier,XGBoost,Neural Network
0,67,72,49,72,67
1,67,72,49,72,64
2,67,70,49,72,68
3,69,72,51,74,67
4,68,72,48,72,67


In [ ]:
## XGBoost 72.5% accuracy, Neural Networks 66%  accuracy which can definitely be improved